## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

In [2]:
# Function to create model, required for KerasClassifier
def create_model():
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #return values
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed=1234)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

,x1,x2,x3,x4,x5,x6,x7,x8,Y
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,0.0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,1.0


In [7]:
# create model
model = KerasClassifier(model=create_model, verbose=1)

In [8]:
model

KerasClassifier(
	model=<function create_model at 0x0000023F8B9251C0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [9]:
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [10]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [11]:
grid_result = grid.fit(X, Y)

Epoch 1/100
39/39 [==============================] - 2s 4ms/step - loss: 12.3935 - accuracy: 0.5482
Epoch 2/100
39/39 [==============================] - 0s 4ms/step - loss: 5.8504 - accuracy: 0.4961
Epoch 3/100
39/39 [==============================] - 0s 4ms/step - loss: 2.9325 - accuracy: 0.5326
Epoch 4/100
39/39 [==============================] - 0s 4ms/step - loss: 2.0665 - accuracy: 0.5638
Epoch 5/100
39/39 [==============================] - 0s 4ms/step - loss: 1.6669 - accuracy: 0.5820
Epoch 6/100
39/39 [==============================] - 0s 4ms/step - loss: 1.4839 - accuracy: 0.5898
Epoch 7/100
39/39 [==============================] - 0s 4ms/step - loss: 1.3610 - accuracy: 0.6120
Epoch 8/100
39/39 [==============================] - 0s 4ms/step - loss: 1.2859 - accuracy: 0.6263
Epoch 9/100
39/39 [==============================] - 0s 4ms/step - loss: 1.2373 - accuracy: 0.6380
Epoch 10/100
39/39 [==============================] - 0s 5ms/step - loss: 1.2018 - accuracy: 0.6276
Epoch 11

In [12]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.696615 using {'batch_size': 20, 'epochs': 100}
0.559896 (0.025780) with: {'batch_size': 10, 'epochs': 10}
0.662760 (0.053115) with: {'batch_size': 10, 'epochs': 50}
0.679688 (0.022326) with: {'batch_size': 10, 'epochs': 100}
0.569010 (0.001841) with: {'batch_size': 20, 'epochs': 10}
0.618490 (0.012890) with: {'batch_size': 20, 'epochs': 50}
0.696615 (0.042473) with: {'batch_size': 20, 'epochs': 100}
0.493490 (0.050463) with: {'batch_size': 40, 'epochs': 10}
0.641927 (0.034987) with: {'batch_size': 40, 'epochs': 50}
0.691406 (0.022999) with: {'batch_size': 40, 'epochs': 100}
0.549479 (0.050865) with: {'batch_size': 60, 'epochs': 10}
0.651042 (0.010253) with: {'batch_size': 60, 'epochs': 50}
0.647135 (0.029635) with: {'batch_size': 60, 'epochs': 100}
0.492188 (0.122327) with: {'batch_size': 80, 'epochs': 10}
0.632812 (0.043848) with: {'batch_size': 80, 'epochs': 50}
0.666667 (0.025780) with: {'batch_size': 80, 'epochs': 100}
0.600260 (0.038976) with: {'batch_size': 100, 'epochs':